## Web scraping

In [1]:
import bs4 as bs
import urllib
import urllib.request
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

In [2]:
HTML = "https://www.trovimap.com/precio-vivienda/barcelona"

In [6]:
# Amb .read_html() podem obtenir fàcilment la taula que ens mostra la web.

df = pd.read_html(HTML)
df[0].head()

,Unnamed: 0,Variación Mensual,Variación 3 meses,Variación anual,€/m2
0,Alella,-0.08%,+0.19%,+2.23%,2.867 €
1,Arenys de Mar,-1.93%,+1.03%,+7.64%,2.287 €
2,Badalona,-0.90%,-1.43%,-2.23%,2.408 €
3,Barberà del Vallès,+2.77%,-1.11%,-2.72%,2.096 €
4,Barcelona,+0.10%,-0.86%,-1.51%,4.059 €


In [7]:
# Amb BeautifulSoup, podrem obtenir tot el String del codi HTML per si volem obtenir més informació.

historypage = urllib.request.urlopen(HTML)
soup = bs.BeautifulSoup(historypage,'html.parser')
makeitastring = ''.join(map(str, soup))

soup
#makeitastring

<!DOCTYPE html>
<html lang="es"><head>
<meta charset="utf-8"/>
<title>Evolución histórica de precios y tendencia del mercado en Barcelona (Provincia)</title>
<base href="/"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://fonts.googleapis.com" rel="dns-prefetch"/>
<link href="https://csi.gstatic.com" rel="dns-prefetch"/>
<link href="https://ssl.google-analytics.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://image-svc.trovimap.com" rel="preconnect"/>
<link as="style" href="https://fonts.googleapis.com/css?family=Source+Sans+Pro:400,600&amp;display=swap" onload="this.onload=null; this.rel='stylesheet';" rel="preload"/>
<script>
    function supportsToken(token) {
        return function(relList){
            if (relList && relList.supports && token) {
                return relList.supports(token);
            }
            return false;
        }
    }

    var supportsPreload = supportsToken("preload");
    var rl = document.createElement("li

In [5]:

#retorna un llistat de ciutats i la seva variació de preu segons el html ja llegit per BeatifulSoup que li hem passat. Aquest html està format segons 
# la provincia. Per exemple: HTML = "https://www.trovimap.com/precio-vivienda/barcelona" retornarà un llistat amb les
# ciutats que es troben en aquesta pàgina que son les ciutats de Barcelona i les seves variacions de preus mensual, els últims 3 mesos
# anual i preu euro/ metre quadrat.

def obtenirCiutatsiVariacio(htmlSoup):
    tableProv = soup.find('table', {'class':'table table-condensed precio-medio-table'})

    tbody = tableProv.find('tbody')

    llistaVarCiutats=[]
    for row in tbody.findAll("tr"):
        cells = row.findAll('td')    
        link = cells[0].find('a')
        ciutat = link.find(text=True)
        #print(ciutat)
        varMensual = cells[1].find(text=True)
        var3mesos = cells[2].find(text=True)
        varAnual = cells[3].find(text=True)
        eumetre = cells[4].find(text=True)
        element=[ciutat,varMensual,var3mesos,varAnual,eumetre]
        llistaVarCiutats.append(element)
        
        
    return llistaVarCiutats


HTML = "https://www.trovimap.com/precio-vivienda/barcelona"

VarCiutat = obtenirCiutatsiVariacio(soup)

print(VarCiutat[0])

Alella
Arenys de Mar
Badalona
Barberà del Vallès
Barcelona
Bigues i Riells
Cabrils
Calella
Canovelles
Castellar del Vallès
Castelldefels
Cerdanyola del Vallès
Corbera de Llobregat
Cornellà de Llobregat
Cubelles
El Masnou
Esparreguera
Gavà
Granollers
Igualada
L'Ametlla del Vallès
L'Hospitalet de Llobregat
Malgrat de Mar
Manresa
Mataró
Mollet del Vallès
Montcada i Reixac
Olesa de Montserrat
Piera
Pineda de Mar
Premià de Dalt
Premià de Mar
Ripollet
Rubí
Sabadell
Santa Coloma de Gramenet
Sant Andreu de Llavaneres
Santa Perpètua de Mogoda
Sant Boi de Llobregat
Sant Celoni
Sant Cugat del Vallès
Sant Pere de Ribes
Sant Quirze del Vallès
Sant Vicenç de Montalt
Sitges
Terrassa
Tordera
Vallirana
Vic
Viladecans
Vilafranca del Penedès
Vilanova del Camí
Vilanova i la Geltrú
Vilassar de Dalt
['Alella', '-0.08%', '+0.19%', '+2.23%', '2.867 €']


In [13]:
HTML = "https://www.trovimap.com/precio-vivienda/"

In [19]:
HTMLEspana = HTML + "espana"

df = pd.read_html(HTMLEspana)
dfProvinciesEspana = df[0]
dfProvinciesEspana = dfProvinciesEspana.rename(columns={"Unnamed: 0": 'Provincies'})

dfProvinciesEspana.head()

,Provincies,Variación Mensual,Variación 3 meses,Variación anual,€/m2
0,A Coruña,+1.48%,-0.16%,-6.86%,1.489 €
1,Albacete,+3.29%,+2.60%,-1.62%,1.262 €
2,Alicante/Alacant,-0.71%,-0.05%,-2.16%,1.722 €
3,Almería,+3.52%,+4.26%,+3.28%,1.193 €
4,Araba/Álava,+0.18%,-0.27%,-4.28%,2.092 €


### Obtenim una taula amb la variació menusal de cada provincia de l'estat.

### Si volem obtenir una taula, tenint per files les ciutats més importants de cada provincia tal com hem fet anteriorment amb la de Barcelona:

In [25]:
dfProvinciesEspana.iloc[0]['Provincies']

'A Coruña'

### Si volem obtenir una taula, tenint per files les ciutats més importants de Catalunta:

In [37]:
# Agafar la taula per cada una de les quatre provincies de Catalunya.
dfBarcelona = pd.read_html(HTML+"barcelona")
dfTarragona = pd.read_html(HTML+"tarragona")
dfGirona = pd.read_html(HTML+"girona")
dfLleida = pd.read_html(HTML+"lleida")
dfBarcelona = dfBarcelona[0]
dfTarragona = dfTarragona[0]
dfGirona = dfGirona[0]
dfLleida = dfLleida[0]

# Concatenar Taules.
dfCatalunya = pd.concat([dfBarcelona, dfTarragona], ignore_index=True)
dfCatalunya = pd.concat([dfCatalunya, dfGirona], ignore_index=True)
dfCatalunya = pd.concat([dfCatalunya, dfLleida], ignore_index=True)

# Renombrar columnes.
dfCatalunya = dfCatalunya.rename(columns={"Unnamed: 0": 'Provincies', "Variación Mensual": 'Variació mensual',
                                "Variación 3 meses": 'Variació tres messos', "Variación anual": 'Variació anual'})

dfCatalunya.tail()

,Provincies,Variació mensual,Variació tres messos,Variació anual,€/m2
93,Tossa de Mar,-2.64%,-3.05%,-2.97%,2.682 €
94,Vidreres,+0.08%,+0.22%,-3.04%,1.423 €
95,Lleida,+1.59%,+1.30%,+6.53%,1.199 €
96,Naut Aran,+1.95%,-,-,3.948 €
97,Vielha e Mijaran,-,-,-0.94%,2.381 €
